# Displaying Tasks from Google Tasks

In [1]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from tkinter import *

In [2]:
SCOPES = ['https://www.googleapis.com/auth/tasks.readonly']

In [5]:
def main():
    creds = None
    
    window = Tk()
    window.title("G33kList Tasks")
    window.geometry("600x400")
    
    scrollbar = Scrollbar(window)
    scrollbar.pack(side=RIGHT, fill=Y
                  )
    text=Text(window, width=60, height=15) 
    text.pack(expand=True)
    
    # Verify hidden credentials folder exists; if not, create it.
    if not os.path.exists('.credentials'):
        os.makedirs('.credentials')
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('.credentials/token.json'):
        creds = Credentials.from_authorized_user_file('.credentials/token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '.credentials/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('.credentials/token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        service = build('tasks', 'v1', credentials=creds)

        # Call the Tasks API
        results = service.tasks().list(tasklist='MTA2ODUxMjg1MDE5MjUzNTMwNjE6MDow', showCompleted=True).execute()
        items = results.get('items', [])

        if not items:
            #print("Dude, there's jack shit to do!")
            text.insert(END, "Dude, there's jack shit to do!")
            return

        #print('Get to work! You gotta do these:')
        text.insert(END, 'Get to work! You gotta do these:\n')
        for item in items:
            # print(u'- {0}'.format(item['title']))
            text.insert(END, ' -  ' + item['title'] + '\n') 
            
        #text.insert(END, 'Dude, you finished doing these:\n')
        #for item in 
        
    except HttpError as err:
        print(err)
        text.insert(END, err + '\n')
        
    def close_win(e):
        window.destroy()

    window.bind('<Escape>', lambda e: close_win(e))

    window.mainloop()

if __name__ == '__main__':
    main()